In [13]:
import os
os.environ["OPENAI_API_KEY"] = "NA" # This satisfies the validator
os.environ["OTEL_SDK_DISABLED"] = "true"
# install the necessary libraries 
# !pip install crewai crewai_tools langchain_community
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [14]:
llm = "ollama/llama3.2"
llm = "ollama/mistral"

In [15]:
from crewai_tools import DirectoryReadTool, FileReadTool
directory_tool = DirectoryReadTool(directory="./knowledge")
file_tool = FileReadTool()



In [16]:
# Create agent with read_knowledge tool (RAG) — knowledge is already saved in the scrape cell above
from crewai import Agent, Task, Crew
# We give the agent both tools: one to list files, one to read them.
data_processor = Agent(
    role='Information Extractor',
    goal='Read file contents accurately and provide summaries without unnecessary tool reuse.',
    backstory="""You are a precise data engineer. When using tools, you MUST provide actual values, not schema definitions.
    
    CRITICAL INSTRUCTIONS FOR TOOL USAGE:
    - When using "Read a file's content" tool, provide the actual file path as a string value.
    - Example CORRECT format: {"file_path": "./knowledge/deeplearning_batch.txt"}
    - Example WRONG format: {"file_path": {"description": "...", "type": "str"}}
    - Do NOT include 'start_line' or 'line_count' parameters unless specifically needed.
    - When using "List files in directory", provide an empty dictionary: {}
    - Always use actual file paths and values, never pass schema definitions or type information.
    - If you see a tool schema description, that is documentation - do NOT pass it as input!
    
    MOST IMPORTANT: After successfully reading a file ONCE, you MUST STOP using tools and provide your Final Answer.
    Do NOT read the same file twice. Do NOT use tools again after you have the file content.""",
    tools=[directory_tool, file_tool],
    llm=llm,
    verbose=True,
    max_iter=5,  # Prevent infinite loops - reduced to force earlier termination
    allow_delegation=False  # Prevent delegation issues
)

In [17]:
# The agent will automatically decide to use the tools based on this description.
analysis_task = Task(
    description="""
    Follow these steps exactly:
    
    1. First, use "List files in directory" tool with Action Input: {}
    
    2. Then, use "Read a file's content" tool ONCE with Action Input: {"file_path": "./knowledge/deeplearning_batch.txt"}
       - The file_path must be the actual path string: "./knowledge/deeplearning_batch.txt"
       - Do NOT include start_line or line_count
       - Do NOT pass {"file_path": {"description": "...", "type": "str"}} - that is WRONG!
       - Only pass the actual file path as a string value
    
    3. CRITICAL STOP CONDITIONS - If you see ANY of these, IMMEDIATELY provide your Final Answer:
       - If you see file content in the Observation (that means you successfully read it)
       - If you see "I tried reusing the same input" (that means you already read the file)
       - If you see the file path listed in the directory output (you have the information you need)
       - After using "Read a file's content" tool ONCE, STOP using tools immediately
    
    4. Provide your Final Answer summarizing the content you read from the file.
       Format: 
       Thought: I now know the final answer
       Final Answer: [your detailed summary here]
    
    IMPORTANT: Once you have read the file content (you'll see it in the Observation), 
    you MUST immediately provide your Final Answer. Do NOT use tools again. 
    Do NOT read the file twice. The file content in the Observation IS your input for the summary.
    """,
    expected_output="A detailed summary of the text within the file.",
    agent=data_processor
)

In [18]:
# 5. Assemble and Kickoff
crew = Crew(
    agents=[data_processor],
    tasks=[analysis_task]
)

result = crew.kickoff()

print("\n--- FINAL CONTEXT SUMMARY ---")
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Information Extractor                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Follow these steps exactly:                                                                                │
│                                                                                                                 │
│      1. First, use "List files in directory" tool with Action Input: {}                                         │
│                                                                                                                 │
│      2. Then, use "Read a file's content" tool ONCE with Action Input: {"file_path":                            │
│  "./knowledge/deeplearning_batch.txt"}                                                                          │
│         - The file_path must be the actual path string: "./knowledge/deeplearning_batch.txt"                    │
│         - Do NOT include start_line or line_count                                                               │
│         - Do NOT pass {"file_path": {"description": "...", "type": "str"}} - that is WRONG!                     │
│         - Only pass the actual file path as a string value                                                      │
│                                                                                                                 │
│      3. CRITICAL STOP CONDITIONS - If you see ANY of these, IMMEDIATELY provide your Final Answer:              │
│         - If you see file content in the Observation (that means you successfully read it)                      │
│         - If you see "I tried reusing the same input" (that means you already read the file)                    │
│         - If you see the file path listed in the directory output (you have the information you need)           │
│         - After using "Read a file's content" tool ONCE, STOP using tools immediately                           │
│                                                                                                                 │
│      4. Provide your Final Answer summarizing the content you read from the file.                               │
│         Format:                                                                                                 │
│         Thought: I now know the final answer                                                                    │
│         Final Answer: [your detailed summary here]                                                              │
│                                                                                                                 │
│      IMPORTANT: Once you have read the file content (you'll see it in the Observation),                         │
│      you MUST immediately provide your Final Answer. Do NOT use tools again.                                    │
│      Do NOT read the file twice. The file content in the Observation IS your input for the summary.             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Information Extractor                                                                                   │
│                                                                                                                 │
│  Thought: Action: List files in directory                                                                       │
│                                                                                                                 │
│  Using Tool: List files in directory                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {}                                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  File paths:                                                                                                    │
│  -./knowledge/deeplearning_batch.txt                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Information Extractor                                                                                   │
│                                                                                                                 │
│  Thought: Thought: Now I will read the content of the provided file using "Read a file's content" tool.         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "file_path": "./knowledge/deeplearning_batch.txt"                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The Batch | DeepLearning.AI | AI News & Insights ✨ New course! Enroll in Agent Skills with Anthropic Explore  │
│  Courses AI Newsletter The Batch Andrew's Letter Data Points ML Research Blog ✨ AI Dev x SF 26 Community       │
│  Forum Events Ambassadors Ambassador Spotlight Resources Membership Start Learning What Matters in AI Right     │
│  Now Subscribe First Name Last Name Where do you live? Select a country What is your role? Please select Keep   │
│  me updated on the latest news, events, and courses Subscribe 🗞️   Stay updated with weekly AI News and          │
│  Insights delivered to your inbox Weekly Issues Andrew's Letters Data Points ML Research Business Science       │
│  Culture Hardware AI Careers About Jan 30, 2026 Agents Go Shopping, Intelligence Redefined, Better Text in      │
│  Pictures, Higher Engagement Means Worse Alignment The Batch AI News and Insights: U.S. policies are driving    │
│  allies away from using American AI technology. Read more Popular Articles Introducing DeepLearning.AI Pro Oct  │
│  31, 2025 1 min read Reinforcement Learning Heats Up, White House Orders Muscular AI Policy, Computer Use       │
│  Gains Momentum, Fine Control of Fine-Tuning Jan 29, 2025 13 min read DeepSeek Sharpens Its Reasoning:          │
│  DeepSeek-R1, an affordable rival to OpenAI’s o1 Jan 22, 2025 4 min read Jan 23, 2026 Self-Driving Reasoning    │
│  Models, ChatGPT Adds Ads, Apple’s Deal with Google, 3D Generation Pronto The Batch AI News and Insights: How   │
│  can businesses go beyond using AI for incremental efficiency gains to create transformative impact? Jan 16,    │
│  2026 Governments vs. Grok, Meta Buys Agent Tech, Healthcare Chatbots, Limits of AI-Powered Retrieval The       │
│  Batch AI News and Insights: Many people are fighting the growth of data centers because they could increase    │
│  CO2 emissions, electricity prices, and water use. Jan 09, 2026 LLMs Go To Confession, Automated Scientific     │
│  Research, What Copilot Users Want, Reasoning For Less The Batch AI News and Insights: We just launched a       │
│  course that shows people who have never coded before, in less than 30 minutes, ...                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Information Extractor                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The content of the file "deeplearning_batch.txt" contains news and insights about DeepLearning.AI, including   │
│  updates on courses, events, community forum, ambassadors, resources, membership, AI research blog, and weekly  │
│  issues of Andrew's Letter. The file also includes popular articles and top stories related to AI for the year  │
│  2025 and 2026, as well as various developments in the field such as reinforcement learning, data centers,      │
│  LLMs, agentic workflows, robotics, and more.                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- FINAL CONTEXT SUMMARY ---
The content of the file "deeplearning_batch.txt" contains news and insights about DeepLearning.AI, including updates on courses, events, community forum, ambassadors, resources, membership, AI research blog, and weekly issues of Andrew's Letter. The file also includes popular articles and top stories related to AI for the year 2025 and 2026, as well as various developments in the field such as reinforcement learning, data centers, LLMs, agentic workflows, robotics, and more.
